# Evaluate

This notebook evaluates the network trained previous notebooks and analyzes the results.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
from datasets.caueeg_script import load_caueeg_task_split
from datasets.pipeline import eeg_collate_fn
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.evaluate import calculate_class_wise_metrics
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0+cu113
cuda is available.


-----

## List up the models to check accuracy

In [4]:
model_names = [
'l8524nml',
'dejxbfxu',
'3874otvp',
'2bw00w05',
'2k8xomy6',
'2452kedi',
'3c9rbdxu',
'2h30v12p',
'1242j086',
'30ek7z1x',
'2apj72km',
'2d7mttdu',
'2615y2jm',
'3jmkkwbc',
'1nq1zwxk',
'2s1700lg',
'3pg95uf1',
'86s3czvy',
'1o7cz2rs',
'52v2mp7f',
'3t0s53ki',
'1cmzq9xq',
'2z7is3m0',
'13i117jq',
'3cy4eb97',
'ph0mix3b',
'cygczhuk',
'gma3iwco',
'1xhv4ae6',
]
model_pool = []

for model_name in model_names:
    path = os.path.join(r'E:\CAUEEG\checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

['lo88puq7',
 '1091v0n5',
 'rwfa415z',
 '1rn23vp7',
 '2r88ber7',
 '1nu3jagp',
 '1mwdhqbz',
 '37bwfmlx',
 '2s17d0vr',
 'ltmd6xgh']


In [5]:
# model_pool = []

# api = wandb.Api()
# runs = api.runs('ipis-mjkim/caueeg-task2-ablation')

# for run in runs:
#     path = os.path.join(r'E:\CAUEEG\checkpoint', run.name, 'checkpoint.pt')
#     try:
#         ckpt = torch.load(path, map_location=device)
#         model_pool.append({'name': run.name, 'path': path})
#     except Exception as e:
#         print(e)
#         print(f'- {run.name}\'s checkpoint cannot be opened: {path}')
        
# pprint.pprint([model_dict['name'] for model_dict in model_pool])

---

## Configurations

In [6]:
base_repeat = 200
verbose = False

-----

## Load and check accuracy

In [7]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
    optimizer = ckpt['optimizer_state']
    scheduler = ckpt['scheduler_state']
    
    print('- checking for', model_dict['name'], config['model'], '...')
    
    # initiate the model
    if '_target_' in config:
        model = hydra.utils.instantiate(config).to(device)
    elif type(config['generator']) is str:
        config['generator'] = getattr(models, config['generator'].split('.')[-1])
        if 'block' in config:
            config['block'] = getattr(models, config['block'].split('.')[-1])
        model = config['generator'](**config).to(device)
    else:
        if 'block' in config:
            if config['block'] == models.resnet_1d.BottleneckBlock1D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_2d.Bottleneck2D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_1d.BasicBlock1D:
                config['block'] = 'basic'
            elif config['block'] == models.resnet_2d.BasicBlock2D:
                config['block'] = 'basic'
                
        model = config['generator'](**config).to(device)
    
    if config.get('ddp', False):
        model_state_ddp = deepcopy(model_state)
        model_state = OrderedDict()
        for k, v in model_state_ddp.items():
            name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
            model_state[name] = v
    
    model.load_state_dict(model_state)
    
    # reconfigure
    config.pop('cwd', 0)
    config['ddp'] = False
    config['crop_timing_analysis'] = False
    config['eval'] = True
    config['device'] = device
    config['run_mode'] = 'eval'
    
    repeat = round(base_repeat / config['crop_multiple'])
    model_dict['repeat'] = repeat
    model_dict['crop_multiple'] = config['crop_multiple']
    model_dict['test_crop_multiple'] = config['test_crop_multiple']    
    
    # build dataset
    _ = build_dataset_for_train(config, verbose=verbose)
    train_loader = _[0]
    val_loader = _[1]
    test_loader = _[2]
    multicrop_test_loader = _[3]
    _, multicrop_val_dataset = load_caueeg_task_split(dataset_path=config['dataset_path'],
                                                      task=config['task'],
                                                      split='val',
                                                      load_event=config['load_event'],
                                                      file_format=config['file_format'],
                                                      transform=config['transform_multicrop'],
                                                      verbose=verbose)
    multicrop_val_loader = DataLoader(multicrop_val_dataset,
                                       batch_size=config['multi_batch_size'],
                                       shuffle=False,
                                       drop_last=False,
                                       num_workers=0,
                                       pin_memory=True,
                                       collate_fn=eeg_collate_fn)
    
    # train accuracy
    train_acc = check_accuracy(model, train_loader, 
                               config['preprocess_test'], config, repeat=repeat)
    model_dict['Train Accuracy'] = train_acc
    
    # val accuracy
    val_acc = check_accuracy(model, val_loader, 
                             config['preprocess_test'], config, repeat=repeat)
    model_dict['Validation Accuracy'] = val_acc

    # Multi-crop val accuracy
    _ = check_accuracy_multicrop_extended(model, multicrop_val_loader, 
                                          config['preprocess_test'], config, repeat=repeat)
    model_dict['Multi-Crop Val Throughput'] = _[4]
    model_dict['Multi-Crop Val Accuracy'] = _[0]
    multi_val_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in multi_val_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'Multi-Crop Val {k} ({c_name})'] = multi_val_class_wise_metrics[k][c]
    
    
    # Test accuracy
    _ = check_accuracy_extended(model, test_loader, 
                                config['preprocess_test'], config, repeat=repeat)
    model_dict['Test Throughput'] = _[4]
    model_dict['Test Accuracy'] = _[0]

    # Multi-crop test accuracy
    _ = check_accuracy_multicrop_extended(model, multicrop_test_loader, 
                                          config['preprocess_test'], config, repeat=repeat)  
    model_dict['Multi-Crop Test Throughput'] = _[4]
    model_dict['Multi-Crop Test Accuracy'] = _[0]
    multi_test_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in multi_test_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'Multi-Crop Test {k} ({c_name})'] = multi_test_class_wise_metrics[k][c]

- checking for lo88puq7 2D-VGG-19 ...
- checking for 1091v0n5 2D-VGG-19 ...
- checking for rwfa415z 2D-VGG-19 ...
- checking for 1rn23vp7 2D-VGG-19 ...
- checking for 2r88ber7 2D-VGG-19 ...
- checking for 1nu3jagp 2D-VGG-19 ...
- checking for 1mwdhqbz 2D-VGG-19 ...
- checking for 37bwfmlx 2D-VGG-19 ...
- checking for 2s17d0vr 2D-VGG-19 ...
- checking for ltmd6xgh 2D-VGG-19 ...


In [8]:
pprint.pprint(model_pool)

[{'Multi-Crop Test Accuracy': 69.8135593220339,
  'Multi-Crop Test Accuracy (Dementia)': 0.8255932203389831,
  'Multi-Crop Test Accuracy (MCI)': 0.7284745762711864,
  'Multi-Crop Test Accuracy (Normal)': 0.8422033898305085,
  'Multi-Crop Test F1-score (Dementia)': 0.566005904681569,
  'Multi-Crop Test F1-score (MCI)': 0.623236124176858,
  'Multi-Crop Test F1-score (Normal)': 0.820166119374155,
  'Multi-Crop Test Precision (Dementia)': 0.8172959805115713,
  'Multi-Crop Test Precision (MCI)': 0.6017257039055404,
  'Multi-Crop Test Precision (Normal)': 0.737921445950643,
  'Multi-Crop Test Recall (Dementia)': 0.43290322580645163,
  'Multi-Crop Test Recall (MCI)': 0.6463414634146342,
  'Multi-Crop Test Recall (Normal)': 0.9230434782608695,
  'Multi-Crop Test Sensitivity (Dementia)': 0.43290322580645163,
  'Multi-Crop Test Sensitivity (MCI)': 0.6463414634146342,
  'Multi-Crop Test Sensitivity (Normal)': 0.9230434782608695,
  'Multi-Crop Test Specificity (Dementia)': 0.9655172413793104,
  'M

In [9]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
        
    model_dict['model'] = config['model']
    model_dict['num_params'] = config.get('num_params', '???')
    model_dict['model size (MiB)'] = sys.getsizeof(pickle.dumps(model_state)) / (1024 * 1024)
    # torch.save(model_state, 'temptemptemp.pt')
    
    model_dict['seq_length'] = config['seq_length']
    model_dict['use_age'] = config['use_age']
    model_dict['photic'] = config['photic']
    model_dict['EKG'] = config['EKG']

    model_dict['awgn'] = config.get('awgn', 0)
    model_dict['awgn_age'] = config.get('awgn_age', 0)
    model_dict['mgn'] = config.get('mgn', 0)
    model_dict['mixup'] = config.get('mixup', 0)
    model_dict['dropout'] = config.get('dropout', 0)
    model_dict['weight_decay'] = config.get('weight_decay', '???')
    model_dict['fc_stages'] = config.get('fc_stages', 1)
    
    model_dict['minibatch'] = round(config['minibatch'])
    model_dict['total_samples'] = round(config.get('total_samples', config['iterations'] * config['minibatch']))
    model_dict['base_lr'] = config.get('base_lr', config.get('LR', '???'))
    model_dict['lr_scheduler_type'] = config.get('lr_scheduler_type', 'constant_with_decay')
    model_dict['warmup_steps'] = config.get('warmup_steps', '???')
    model_dict['seed'] = config.get('seed', '???')

In [10]:
pd.DataFrame(model_pool)

,name,path,repeat,crop_multiple,test_crop_multiple,Train Accuracy,Validation Accuracy,Multi-Crop Val Accuracy,Multi-Crop Val Throughput,Multi-Crop Val Accuracy (Normal),...,mixup,dropout,weight_decay,fc_stages,minibatch,total_samples,base_lr,lr_scheduler_type,warmup_steps,seed
0,lo88puq7,E:\CAUEEG\checkpoint\lo88puq7\checkpoint.pt,50,4,8,100.000000,64.289916,66.386555,298.853588,0.772269,...,0.3,0.201491,0.012529,2,128,100000000,0.000085,constant_with_decay,39062,0
1,1091v0n5,E:\CAUEEG\checkpoint\1091v0n5\checkpoint.pt,50,4,8,100.000000,62.592437,64.067227,293.288767,0.751765,...,0.3,0.201491,0.012529,2,128,100000000,0.000085,constant_with_decay,39062,0
2,rwfa415z,E:\CAUEEG\checkpoint\rwfa415z\checkpoint.pt,50,4,8,100.000000,60.428571,61.445378,288.015088,0.744706,...,0.3,0.201491,0.012529,2,128,100000000,0.000085,constant_with_decay,39062,0
3,1rn23vp7,E:\CAUEEG\checkpoint\1rn23vp7\checkpoint.pt,50,4,8,100.000000,64.777311,66.890756,299.556455,0.773277,...,0.3,0.201491,0.012529,2,128,100000000,0.000085,constant_with_decay,39062,0
4,2r88ber7,E:\CAUEEG\checkpoint\2r88ber7\checkpoint.pt,50,4,8,99.998922,64.869748,67.126050,298.653713,0.815126,...,0.3,0.201491,0.012529,2,128,100000000,0.000085,constant_with_decay,39062,0
5,1nu3jagp,E:\CAUEEG\checkpoint\1nu3jagp\checkpoint.pt,50,4,8,100.000000,64.781513,66.184874,299.062617,0.774118,...,0.0,0.201491,0.012529,2,128,100000000,0.000085,constant_with_decay,39062,0
6,1mwdhqbz,E:\CAUEEG\checkpoint\1mwdhqbz\checkpoint.pt,50,4,8,100.000000,60.420168,61.445378,299.003895,0.770756,...,0.0,0.000000,0.012529,2,128,100000000,0.000085,constant_with_decay,39062,0
7,37bwfmlx,E:\CAUEEG\checkpoint\37bwfmlx\checkpoint.pt,50,4,8,100.000000,61.588235,62.655462,278.599842,0.759328,...,0.0,0.000000,0.012529,1,128,100000000,0.000085,constant_with_decay,39062,0
8,2s17d0vr,E:\CAUEEG\checkpoint\2s17d0vr\checkpoint.pt,50,4,8,99.998922,62.563025,64.857143,276.984760,0.778655,...,0.0,0.000000,0.012529,1,128,100000000,0.000085,constant_with_decay,39062,0
9,ltmd6xgh,E:\CAUEEG\checkpoint\ltmd6xgh\checkpoint.pt,50,4,8,100.000000,54.512605,56.084034,286.722950,0.703193,...,0.0,0.000000,0.012529,1,128,100000000,0.000085,constant_with_decay,39062,0


In [11]:
pd.DataFrame(model_pool).to_csv('local/output/caueeg-task2-ablations-summary.csv')